In [1]:
from sklearn.model_selection import train_test_split
from collections import Counter
from tqdm import tqdm_notebook

import pandas as pd
import random
import nltk

import config
import log

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/colan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### Loading in Data

In [3]:
log.info("Loading Training Data")

# use a low n-rows when testing else don't set it
df = pd.read_csv(config.TRAIN_PATH, header=0, nrows=1000, usecols=['target', 'comment_text'])
# df = pd.read_csv(config.TRAIN_PATH, header=0, usecols=['target', 'comment_text'])

INFO: Loading Training Data


In [4]:
df

,target,comment_text
0,0.000000,"This is so cool. It's like, 'would you want yo..."
1,0.000000,Thank you!! This would make my life a lot less...
2,0.000000,This is such an urgent design problem; kudos t...
3,0.000000,Is this something I'll be able to install on m...
4,0.893617,haha you guys are a bunch of losers.
...,...,...
995,0.000000,I call dibs on the Tonya Harding pic
996,0.629630,This is just freaking sad. Another great place...
997,0.000000,This is a test comment (trying out the new Civ...
998,0.000000,This is a test comment (trying out the new Civ...


### Simplifying Targets

For this sample we are going to only use the input text and have it guess a sample label. To get the sample label we are going to create 3 intermediaries for the calculated toxicity target betwee 0 and 1.

* 0.00-0.20 -> not toxic
* 0.21-0.60 -> mildly toxic
* 0.61-1.00 -> toxic

In [5]:
targets = df['target'].apply(lambda t: 'not_toxic' if t <= 0.2 else 'mildly_toxic' if t <= 0.6 else 'toxic')
df.update(targets)

In [6]:
df.target.value_counts()

not_toxic       893
mildly_toxic     82
toxic            25
Name: target, dtype: int64

### Random Classifier

In [7]:
class WeightedRandomClassifier():
    def __init__(self, counts):
        total = float(sum([count for count in counts]))
        
        self.probabilities = []
        self.probabilities.append(counts.not_toxic / total)
        self.probabilities.append(counts.mildly_toxic / total)        
        self.probabilities.append(counts.toxic / total)  
        
        self.classes = ['not_toxic', 'mildly_toxic', 'toxic']
        
    def predict(self):
        # https://stackoverflow.com/questions/13343347/randomizing-two-lists-and-maintaining-order-in-python
        combined = list(zip(self.probabilities, self.classes))
        random.shuffle(combined)
        self.probabilities, self.classes = zip(*combined)
        
        # run until a class is found
        probability_to_reach = random.random()
        current_probability = 0
        index = 0
        
        while True:
            current_probability += self.probabilities[index]
            
            if current_probability < probability_to_reach:
                break
                
            index += 1
            
            if index == len(self.classes):
                index -= 1
                break
            
        return self.classes[index]
    
    def accuracy(self, nltk_expected_input):
        '''
        [(input, target_class), ...]
        '''
        correct = 0
        total = 0
        
        for element in nltk_expected_input:
            if self.predict() == element[1]:
                correct += 1
            
            total += 1
            
        return correct / total

In [8]:
class RandomClassifier():
    def __init__(self):
        self.classes = ['not_toxic', 'mildly_toxic', 'toxic']

    def predict(self):
        return random.choice(self.classes)

    def accuracy(self, nltk_expected_input):
        '''
        [(input, target_class), ...]
        '''
        correct = 0
        total = 0

        for element in nltk_expected_input:
            if self.predict() == element[1]:
                correct += 1

            total += 1

        return correct / total

In [9]:
weighted_random_classifier = WeightedRandomClassifier(df.target.value_counts())
random_classifier = RandomClassifier()

### Removing Case

In [10]:
targets = df['comment_text'].apply(lambda comment: comment.lower())
df.update(targets)

In [11]:
df

,target,comment_text
0,not_toxic,"this is so cool. it's like, 'would you want yo..."
1,not_toxic,thank you!! this would make my life a lot less...
2,not_toxic,this is such an urgent design problem; kudos t...
3,not_toxic,is this something i'll be able to install on m...
4,toxic,haha you guys are a bunch of losers.
...,...,...
995,not_toxic,i call dibs on the tonya harding pic
996,toxic,this is just freaking sad. another great place...
997,not_toxic,this is a test comment (trying out the new civ...
998,not_toxic,this is a test comment (trying out the new civ...


### Splitting

We split the data so we can estimate the performace of our algorithm. You can set the switch value below to decide the split. Default is to 0.8 where 80% of the training is trained on and 20% is tested on.

In [12]:
split_percentage = 0.8

In [13]:
training_set, testing_set = train_test_split(df, test_size = 1.0 - split_percentage)

log.info(f'Train Data Size: {len(training_set)}')
log.info(f'Test Data Size: {len(testing_set)}')

INFO: Train Data Size: 800
INFO: Test Data Size: 200


### Formatting and Bag of Words

We'll use the 1000 most occurring words for our bag of words in this example.

In [14]:
training_data = []
training_data = []

word_counts = Counter()

In [15]:
train = training_set.to_dict()
test = testing_set.to_dict()

In [16]:
tweet_tokenizer = nltk.tokenize.TweetTokenizer()

for key in tqdm_notebook(train['comment_text']):
    tokens = tweet_tokenizer.tokenize(train['comment_text'][key])
    train['comment_text'][key] = tokens
    
    c = Counter(tokens)
    word_counts.update(c)

In [17]:
most_common_words = word_counts.most_common(1000)

In [18]:
log.info('training set updates')

for key in tqdm_notebook(train['comment_text']):
    tokenized_sentence = train['comment_text'][key]
    bow = {}
    
    for word in most_common_words:
        word = word[0]
        
        if word in tokenized_sentence:
            bow[word] = tokenized_sentence.count(word)
        else:
            bow[word] = 0
            
    train['comment_text'][key] = bow

INFO: training set updates


In [19]:
log.info('testing set updates')

for key in tqdm_notebook(test['comment_text']):
    tokenized_sentence = tweet_tokenizer.tokenize(test['comment_text'][key])
    bow = {}
    
    for word in most_common_words:
        word = word[0]
        
        if word in tokenized_sentence:
            bow[word] = tokenized_sentence.count(word)
        else:
            bow[word] = 0

    test['comment_text'][key] = bow

INFO: testing set updates


In [20]:
log.info('formatting training data for NLTK')

final_training_set = []
for key in tqdm_notebook(train['target']):
    final_training_set.append((train['comment_text'][key], train['target'][key]))

INFO: formatting training data for NLTK


In [21]:
log.info('formatting testing data for nltk')

final_testing_set = []
for key in tqdm_notebook(test['target']):
    final_testing_set.append((test['comment_text'][key], test['target'][key]))

INFO: formatting testing data for nltk


### Training

In [22]:
naive_bayes_classifier = nltk.NaiveBayesClassifier.train(final_training_set)

### Evaluation

In [23]:
print(f'Random Classifier: {random_classifier.accuracy(final_testing_set)}')
print(f'Weighted Random Classifier: {weighted_random_classifier.accuracy(final_testing_set)}')
print(f'Niave Bayes Accuracy: {nltk.classify.accuracy(naive_bayes_classifier, final_testing_set)}')
print(f'Precision: not implemented')
print(f'Recall: not implemented')

Random Classifier: 0.33
Weighted Random Classifier: 0.12
Niave Bayes Accuracy: 0.795
Precision: not implemented
Recall: not implemented


In [24]:
naive_bayes_classifier.show_most_informative_features(100)

Most Informative Features
                   bunch = 1               toxic : not_to =     75.6 : 1.0
                    guys = 1               toxic : not_to =     32.4 : 1.0
                   going = 2               toxic : not_to =     31.7 : 1.0
                    want = 3               toxic : not_to =     31.0 : 1.0
                   after = 2               toxic : not_to =     31.0 : 1.0
                       * = 1               toxic : not_to =     30.4 : 1.0
                     law = 2               toxic : not_to =     29.8 : 1.0
                     few = 2               toxic : not_to =     19.0 : 1.0
                       ; = 2               toxic : not_to =     18.6 : 1.0
                   mayor = 2              mildly : not_to =     17.4 : 1.0
                  threat = 1              mildly : not_to =     17.4 : 1.0
                   armed = 1              mildly : not_to =     15.0 : 1.0
                   total = 1               toxic : not_to =     13.9 : 1.0